<div dir="rtl">

# ۲ – ساخت RAG ساده روی اخبار فارسی

در این نوت‌بوک، زیرمجموعه‌ی داده‌ی آماده‌شده را به چند بخش کوچک (چانک) تبدیل می‌کنید، برای هر چانک بردارهای embedding می‌سازید، یک ایندکس برداری (مثلاً با FAISS) ایجاد می‌کنید و در نهایت یک سیستم RAG ساده برای پاسخ‌گویی به سؤال‌های فارسی پیاده‌سازی می‌کنید.

</div>


In [13]:
import pandas as pd
import os
import re
import pickle
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi

In [2]:
df = pd.read_csv("../data/processed/news_subset.csv")

print(df.head())

print("shape:", df.shape)
print("columns:", df.columns.tolist())


       id                                              title  \
0    7910  پرداخت 10 میلیارد ریال تسهیلات گردشگری در زنجا...   
1  144943  حضور 3500 نفر در اردوهای هفت اقلیم در استان مرکزی   
2  229417  آموزش سلامت دوران بلوغ در مدارس عشایر اصفهان ب...   
3  149383  کیفیت فعلی هوای مشهد ناسالم است/ سال جاری برای...   
4  103531  12 هزار و 920 همسر شهید یا جانباز در زنجان زند...   

                  short_link  service     subgroup  \
0       http://fna.ir/f1x73c  استانها        زنجان   
1        http://fna.ir/491pa  استانها        مرکزی   
2  http://mehrnews.com/xZV7j  استانها       اصفهان   
3        http://fna.ir/4b6gc  استانها  خراسان رضوی   
4        http://fna.ir/2iolx  استانها        زنجان   

                                            abstract  \
0  مدیرکل میراث فرهنگی، صنایع دستی و گردشگری استا...   
1  فرمانده سپاه روح الله استان مرکزی گفت: از ابتد...   
2  اصفهان-مشاور امور بانوان مدیرکل آموزش و پرورش ...   
3  شاخص کیفیت هوای مشهد طی ۲۴ ساعت گذشته بر روی ع...   
4  مسوول ک

In [3]:
def chunk_text(text: str, chunk_size: int = 800, overlap: int = 150):
    text = ("" if text is None else str(text)).strip()
    step = chunk_size - overlap
    return [text[i:i + chunk_size] for i in range(0, len(text), step) if text[i:i + chunk_size].strip()]

df = pd.read_csv("../data/processed/news_subset.csv")

for col in ["title", "abstract", "body"]:
    df[col] = df[col].fillna("").astype(str).str.strip()

df["text_for_chunk"] = (
    "عنوان: " + df["title"] + "\n"
    "خلاصه: " + df["abstract"] + "\n"
    "متن: " + df["body"]
).str.strip()

rows = []
for _, r in df.iterrows():
    doc_id = r["id"]
    category = r["service"] if pd.notna(r["service"]) else r["subgroup"]
    date = r["published_datetime"]
    chunks = chunk_text(r["text_for_chunk"], 800, 150)
    for i, ch in enumerate(chunks):
        rows.append({"chunk_id": f"{doc_id}_{i}", "doc_id": doc_id, "text": ch, "category": category, "date": date})

chunks_df = pd.DataFrame(rows, columns=["chunk_id", "doc_id", "text", "category", "date"])
print(chunks_df.shape)
print(chunks_df.head(5))


(49959, 5)
   chunk_id  doc_id                                               text  \
0    7910_0    7910  عنوان: پرداخت 10 میلیارد ریال تسهیلات گردشگری ...   
1    7910_1    7910  ر آسیب شده اند، تمدید شده است. وی با اشاره به ...   
2    7910_2    7910  افزود: به همین علت موزه های استان زنجان دوباره...   
3    7910_3    7910   و واحدهای پذیرایی بین راهی با رعایت پروتکل ها...   
4  144943_0  144943  عنوان: حضور 3500 نفر در اردوهای هفت اقلیم در ا...   

  category                 date  
0  استانها  2021-01-13 06:38:00  
1  استانها  2021-01-13 06:38:00  
2  استانها  2021-01-13 06:38:00  
3  استانها  2021-01-13 06:38:00  
4  استانها  2021-10-06 04:04:03  


In [ ]:
out_dir = "artifacts"
os.makedirs(out_dir, exist_ok=True)

meta_cols = [c for c in ["chunk_id", "text", "category", "date"] if c in chunks_df.columns]
meta = chunks_df[meta_cols].copy()
meta.to_csv(os.path.join(out_dir, "chunks_meta.csv"), index=False, encoding="utf-8-sig")

texts = meta["text"].astype(str).tolist()

def tokenize_fa(s: str):
    s = "" if s is None else str(s)
    s = s.lower()
    s = re.sub(r"[^\w\u0600-\u06FF]+", " ", s)
    toks = [t for t in s.split() if t]
    return toks

semantic_model_name = "paraphrase-multilingual-MiniLM-L12-v2"
semantic_model = SentenceTransformer(semantic_model_name)

sem = semantic_model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True).astype(np.float32)
sem = normalize(sem, norm="l2").astype(np.float32)

index_sem = faiss.IndexFlatIP(sem.shape[1])
index_sem.add(sem)
faiss.write_index(index_sem, os.path.join(out_dir, "faiss_sem.index"))

corpus_tokens = [tokenize_fa(t) for t in texts]
bm25 = BM25Okapi(corpus_tokens)
with open(os.path.join(out_dir, "bm25.pkl"), "wb") as f:
    pickle.dump(bm25, f)

with open(os.path.join(out_dir, "config.pkl"), "wb") as f:
    pickle.dump({"semantic_model": semantic_model_name}, f)

print("saved:", os.path.abspath(out_dir))
print("faiss_sem.index ntotal:", index_sem.ntotal)
print("bm25 docs:", len(corpus_tokens))

meta_loaded = pd.read_csv(os.path.join(out_dir, "chunks_meta.csv"))
index_sem_loaded = faiss.read_index(os.path.join(out_dir, "faiss_sem.index"))
with open(os.path.join(out_dir, "bm25.pkl"), "rb") as f:
    bm25_loaded = pickle.load(f)
with open(os.path.join(out_dir, "config.pkl"), "rb") as f:
    cfg = pickle.load(f)
semantic_model_loaded = SentenceTransformer(cfg["semantic_model"])

def _semantic_search(question, top_k):
    q = semantic_model_loaded.encode([question], convert_to_numpy=True).astype(np.float32)
    q = normalize(q, norm="l2").astype(np.float32)
    scores, idxs = index_sem_loaded.search(q, top_k)
    return idxs[0].tolist(), scores[0].tolist()

def _bm25_search(question, top_k):
    q_tokens = tokenize_fa(question)
    scores = np.asarray(bm25_loaded.get_scores(q_tokens), dtype=np.float32)
    if top_k >= len(scores):
        top_idx = np.argsort(-scores)
    else:
        top_idx = np.argpartition(scores, -top_k)[-top_k:]
        top_idx = top_idx[np.argsort(-scores[top_idx])]
    return top_idx.tolist(), scores[top_idx].tolist()




Batches: 100%|██████████| 781/781 [11:02<00:00,  1.18it/s]


saved: d:\Programming\Bootcamp\دوره ها\RAG2\rag-persian-news-lite-main\notebooks\artifacts
faiss_sem.index ntotal: 49959
bm25 docs: 49959


In [ ]:
index = faiss.read_index("artifacts/faiss_sem.index")

D = index.d
ntotal = index.ntotal
print("D:", D, "| ntotal:", ntotal)

q = np.random.rand(1, D).astype(np.float32)
q /= (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)

scores, idxs = index.search(q, 5)
print("idxs:", idxs[0].tolist())
print("scores:", scores[0].tolist())


D: 384 | ntotal: 49959
idxs: [6257, 7649, 24502, 20634, 40390]
scores: [0.06989748775959015, 0.06839863955974579, 0.06826449930667877, 0.06683947890996933, 0.0666075199842453]


In [8]:
def retrieve(question, top_k=5, embedding_model="semantic", fusion_k=60, pool_mult=20):
    if embedding_model == "semantic":
        idxs, _ = _semantic_search(question, top_k)
        return meta_loaded.iloc[idxs]["text"].tolist()

    if embedding_model == "bm25":
        idxs, _ = _bm25_search(question, top_k)
        return meta_loaded.iloc[idxs]["text"].tolist()

    if embedding_model == "combined":
        pool_k = min(len(meta_loaded), top_k * pool_mult)
        sem_idxs, _ = _semantic_search(question, pool_k)
        bm_idxs, _ = _bm25_search(question, pool_k)

        fused = {}
        for r, i in enumerate(sem_idxs, start=1):
            fused[i] = fused.get(i, 0.0) + 1.0 / (fusion_k + r)
        for r, i in enumerate(bm_idxs, start=1):
            fused[i] = fused.get(i, 0.0) + 1.0 / (fusion_k + r)

        best = sorted(fused.items(), key=lambda x: x[1], reverse=True)[:top_k]
        idxs = [i for i, _ in best]
        return meta_loaded.iloc[idxs]["text"].tolist()

    raise ValueError("embedding_model باید یکی از این‌ها باشد: 'semantic' یا 'bm25' یا 'combined'")

In [9]:
def answer(question, top_k=5, embedding_model="semantic"):
    chunks = retrieve(question, top_k=top_k, embedding_model=embedding_model)
    return "\n\n".join(chunks)


In [11]:
questions = [
    "آخرین خبر درباره قیمت دلار چیست؟",
    "در مورد مذاکرات هسته‌ای چه خبرهایی منتشر شده؟",
]

for q in questions:
    print("Q:", q)

    print("\n--- semantic ---")
    print(answer(q, top_k=5, embedding_model="semantic"))

    print("\n--- bm25 ---")
    print(answer(q, top_k=5, embedding_model="bm25"))

    print("\n--- combined ---")
    print(answer(q, top_k=5, embedding_model="combined"))

    print("\n" + "="*70 + "\n")


Q: آخرین خبر درباره قیمت دلار چیست؟

--- semantic ---
عنوان: صندوق بین المللی پول: روند کنار گذاشتن دلار آغاز شده است
خلاصه: رئیس صندوق بین المللی پول می گوید که روند فاصله گرفتن از دلار آمریکا هم اکنون در جریان است اما هنوز جایگزینی برای اسکناس سبز آمریکا وجود ندارد.
متن: به گزارش خبرگزاری مهر به نقل از اسپوتنیک، کریستینا جورجیوا، مدیر اجرایی صندوق بین المللی پول بر این باور است که فاصله گرفتن از دلار آمریکا به عنوان ذخیره ارزی جهان در حال وقوع است اما به گفته وی در آینده نزدیک جایگزینی برای این ارز وجود ندارد. جورجیوا درباره این مساله گفت: دلیل نقش آفرینی دلار، قدرت اقتصاد آمریکا و عمق بازارهای سرمایه در این کشور است. وی افزود: و اگر در فکر گزینه جایگزین هستید، آن هم در دنیایی که احتمالا به طور گسترده به سمت ارزهای دیجیتال بانک مرکزی کشورها حرکت می کند، شاید گزینه های متفاوتی روی میز باشد اما در نهایت، قدرت اقتصاد آمریکا، عمق بازار سرمایه و

 شد و ما شاهد جنگ اقتصادی غرب به رهبری آمریکا علیه کشورمان بودیم قیمت دلار بسیار بالاتر از 5 هزار تومان بود. وی افزود: در پایان دولت دهم قیمت دل